# Data Cleaning

SC/MATH 1130 A - Fall 2025

The Impact of AI on Canadian Wages


In [45]:
import pandas as pd

from pathlib import Path

import warnings

warnings.filterwarnings("ignore")

In [46]:
# set up folder paths
BASE_DIR = Path("..")
RAW_DATA = BASE_DIR / "data" / "raw"
CLEANED_DATA = BASE_DIR / "data" / "cleaned"
MERGED_DATA = BASE_DIR / "data" / "merged"
METADATA = BASE_DIR / "data" / "metadata"

# create output folders if they don't exist yet
CLEANED_DATA.mkdir(parents=True, exist_ok=True)
MERGED_DATA.mkdir(parents=True, exist_ok=True)
METADATA.mkdir(parents=True, exist_ok=True)

print("folders ready")

folders ready


## Step 1: Load wage data (2012-2025)

Loading all 14 wage files and combining them into one big dataset.


In [47]:
# load all wage files from 2012-2025
wage_files = sorted((RAW_DATA / "wages").glob("wages_*.csv"))
print(f"found {len(wage_files)} files")

dfs: list[pd.DataFrame] = []
for f in wage_files:
    # using latin-1 encoding because files have French characters (é, etc)
    df = pd.read_csv(f, encoding="latin-1")
    dfs.append(df)

df_wages = pd.concat(dfs, ignore_index=True)

# convert Reference_Period to numeric (some files have it as string)
df_wages["Reference_Period"] = pd.to_numeric(df_wages["Reference_Period"], errors="coerce")

print(f"total rows: {len(df_wages):,}")
print(f"total columns: {len(df_wages.columns)}")
print(f"years: {int(df_wages['Reference_Period'].min())} to {int(df_wages['Reference_Period'].max())}")
print(f"unique occupations: {df_wages['NOC_CNP'].nunique():,}")

found 14 files
total rows: 613,008
total columns: 52
years: 2006 to 2024
unique occupations: 500


In [48]:
df_wages.head()

,ï»¿NOC_CNP_2006,NOC_Title,Titre_CNP,PROV,ER_Code_Code_RE,ER_Name_Nom_RE,Low_Wage_Salaire_Minium,Median_Wage_Salaire_Median,High_Wage_Salaire_Maximal,Data_Source_E,...,prov,ER_Name,Nom_RE,NOC_Title_eng,NOC_Title_fra,Quartile1_Wage_Salaire_Quartile1,Quartile3_Wage_Salaire_Quartile3,Non_WageBen_pct,Source2025_NHQ,EmployeesWithNonWageBenefit_Pct
0,11,Legislators,Membres des corps lÃ©gislatifs,National,ER00,National,8.17,28.00,49.45,Labour Force Survey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,Legislators,Membres des corps lÃ©gislatifs,Newfoundland and Labrador,ER10,Newfoundland and Labrador,17.95,40.06,64.97,Labour Force Survey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,Legislators,Membres des corps lÃ©gislatifs,Newfoundland and Labrador,ER1010,Avalon Peninsula,19.23,45.67,66.67,Labour Force Survey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,Legislators,Membres des corps lÃ©gislatifs,Newfoundland and Labrador,ER1020,South Coast--Burin Peninsula,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Legislators,Membres des corps lÃ©gislatifs,Newfoundland and Labrador,ER1030,West Coast--Northern Peninsula--Labrador,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 2: Clean and standardize wage data

First we'll check how much data is missing (Stats Canada hides data for privacy), then convert all wages to hourly rates and clean up job titles.


In [49]:
# check how much data is missing
# (Stats Canada suppresses data for privacy when sample size is too small)
wage_cols = [
    "Low_Wage_Salaire_Minium",
    "Median_Wage_Salaire_Median",
    "High_Wage_Salaire_Maximal",
    "Average_Wage_Salaire_Moyen",
    "Quartile1_Wage_Salaire_Quartile1",
    "Quartile3_Wage_Salaire_Quartile3",
]

print("checking missing wage data...")
for col in wage_cols:
    missing = df_wages[col].isnull().sum()
    pct = missing / len(df_wages) * 100
    print(f"{col}: {missing:,} missing ({pct:.1f}%)")

checking missing wage data...
Low_Wage_Salaire_Minium: 385,834 missing (62.9%)
Median_Wage_Salaire_Median: 383,394 missing (62.5%)
High_Wage_Salaire_Maximal: 385,814 missing (62.9%)
Average_Wage_Salaire_Moyen: 542,612 missing (88.5%)
Quartile1_Wage_Salaire_Quartile1: 580,638 missing (94.7%)
Quartile3_Wage_Salaire_Quartile3: 580,634 missing (94.7%)


In [50]:
# some wages are annual, some are hourly - need to make them all hourly
# 2080 hours = 40 hours/week * 52 weeks/year
HOURS_PER_YEAR = 2080

# mapping of old column names to new hourly column names
wage_mapping = {
    "Low_Wage_Salaire_Minium": "Low_Wage_Hourly",
    "Median_Wage_Salaire_Median": "Median_Wage_Hourly",
    "High_Wage_Salaire_Maximal": "High_Wage_Hourly",
    "Average_Wage_Salaire_Moyen": "Average_Wage_Hourly",
    "Quartile1_Wage_Salaire_Quartile1": "Quartile1_Wage_Hourly",
    "Quartile3_Wage_Salaire_Quartile3": "Quartile3_Wage_Hourly",
}

for old_col, new_col in wage_mapping.items():
    df_wages[new_col] = df_wages[old_col].copy()

    # convert annual to hourly where flag = 1
    annual_rows = df_wages["Annual_Wage_Flag_Salaire_annuel"] == 1
    df_wages.loc[annual_rows, new_col] = df_wages.loc[annual_rows, old_col] / HOURS_PER_YEAR

# show how many rows needed conversion
annual_count = (df_wages["Annual_Wage_Flag_Salaire_annuel"] == 1).sum()
print(f"converted {annual_count:,} annual wages to hourly (÷ 2080)")

converted 9,739 annual wages to hourly (÷ 2080)


In [51]:
# flag rows that have no wage data at all
hourly_cols = [
    "Low_Wage_Hourly",
    "Median_Wage_Hourly",
    "High_Wage_Hourly",
    "Average_Wage_Hourly",
    "Quartile1_Wage_Hourly",
    "Quartile3_Wage_Hourly",
]
df_wages["missing_wage_flag"] = df_wages[hourly_cols].isnull().all(axis=1).astype(int)

print(f"rows with no wage data: {df_wages['missing_wage_flag'].sum():,}")

rows with no wage data: 383,375


In [52]:
# clean up job titles
df_wages["NOC_Title_Standardized"] = df_wages["NOC_Title_eng"].str.strip().str.title()

# merge similar titles (e.g. software engineer and software developer are the same)
replacements = {
    "Software Engineer": "Software Developer",
    "Computer Programmer": "Software Developer",
    "Web Designer": "Web Developer",
}

for old, new in replacements.items():
    mask = df_wages["NOC_Title_Standardized"].str.contains(old, case=False, na=False)
    df_wages.loc[mask, "NOC_Title_Standardized"] = new

print(f"unique titles before: {df_wages['NOC_Title_eng'].nunique():,}")
print(f"unique titles after: {df_wages['NOC_Title_Standardized'].nunique():,}")

unique titles before: 516
unique titles after: 515


In [53]:
# standardize province and region names
# provinces: 10 provinces + 3 territories + NAT (national) + regional codes
df_wages["Province"] = df_wages["prov"].str.strip().str.upper()
df_wages["Region"] = df_wages["ER_Name"].str.strip()

print(f"provinces: {df_wages['Province'].nunique()}")
print(f"province codes: {sorted(df_wages['Province'].dropna().unique())}")
print("\nAll provinces:")
print(df_wages["Province"].value_counts(dropna=False))

provinces: 16
province codes: ['AB', 'BC', 'MB', 'NAT', 'NB', 'NL', 'NS', 'NT', 'NU', 'NWT', 'ON', 'PE', 'PEI', 'QC', 'SK', 'YK']

All provinces:
Province
NaN    480396
QC      27864
ON      18576
BC      13932
AB      13932
MB      13932
SK      10836
NB       9288
NS       9288
NL       7740
YK       1548
NU       1548
NAT      1032
PEI      1032
NWT      1032
PE        516
NT        516
Name: count, dtype: int64


In [54]:
# flag entry level jobs
# approach: if a job's median wage is in the bottom 25% of all wages, it's probably entry level

df_wages["is_entry_level"] = 0

# get the overall 25th percentile of median wages
overall_q25 = df_wages["Median_Wage_Hourly"].quantile(0.25)

# flag jobs where median wage <= 25th percentile
entry_mask = df_wages["Median_Wage_Hourly"].notna() & (df_wages["Median_Wage_Hourly"] <= overall_q25)
df_wages.loc[entry_mask, "is_entry_level"] = 1

print(f"entry level jobs: {df_wages['is_entry_level'].sum():,}")
print(f"overall 25th percentile wage: ${overall_q25:.2f}/hr")

entry level jobs: 57,444
overall 25th percentile wage: $19.63/hr


In [55]:
# keep only the columns we need
cols = [
    "NOC_CNP",
    "NOC_Title_Standardized",
    "Province",
    "Region",
    "Reference_Period",
    "Annual_Wage_Flag_Salaire_annuel",
    "Low_Wage_Hourly",
    "Median_Wage_Hourly",
    "High_Wage_Hourly",
    "Average_Wage_Hourly",
    "Quartile1_Wage_Hourly",
    "Quartile3_Wage_Hourly",
    "missing_wage_flag",
    "is_entry_level",
]

df_wages_clean = df_wages[cols].copy()
print(f"cleaned wage data: {df_wages_clean.shape}")
print(f"rows with wage data: {(df_wages_clean['missing_wage_flag'] == 0).sum():,}")
df_wages_clean.head()

cleaned wage data: (613008, 14)
rows with wage data: 229,633


,NOC_CNP,NOC_Title_Standardized,Province,Region,Reference_Period,Annual_Wage_Flag_Salaire_annuel,Low_Wage_Hourly,Median_Wage_Hourly,High_Wage_Hourly,Average_Wage_Hourly,Quartile1_Wage_Hourly,Quartile3_Wage_Hourly,missing_wage_flag,is_entry_level
0,NaN,NaN,NaN,NaN,NaN,0.0,8.17,28.00,49.45,NaN,NaN,NaN,0,0
1,NaN,NaN,NaN,NaN,NaN,0.0,17.95,40.06,64.97,NaN,NaN,NaN,0,0
2,NaN,NaN,NaN,NaN,NaN,0.0,19.23,45.67,66.67,NaN,NaN,NaN,0,0
3,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0


## Step 3: Load and clean AI adoption data

We have two datasets: businesses _planning_ to use AI (Q3 2024) and businesses _actually using_ AI (Q2 2025).


In [56]:
# AI planned adoption (Q3 2024)
df_ai_plan = pd.read_csv(RAW_DATA / "33100878-eng" / "ai_planned_q3_2024.csv", encoding="utf-8")
# clean column names (remove quotes if any)
df_ai_plan.columns = df_ai_plan.columns.str.strip().str.replace('"', "")
print(f"AI planned: {df_ai_plan.shape}")

AI planned: (19698, 16)


In [57]:
# simplify the long column name
ai_col = "Use of artificial intelligence (AI) by businesses or organizations in producing goods or delivering services over the next 12 months"

df_ai_plan["AI_Status"] = df_ai_plan[ai_col].apply(
    lambda x: "Yes" if "Yes" in str(x) else ("No" if "No" in str(x) else "Unknown")
)

df_ai_plan["Industry"] = df_ai_plan["Business characteristics"].str.strip()

# keep only yes responses
df_ai_plan = df_ai_plan[df_ai_plan["AI_Status"] == "Yes"][["REF_DATE", "GEO", "Industry", "AI_Status", "VALUE"]].copy()

df_ai_plan["Source"] = "Planned_Q3_2024"
print(f"cleaned: {df_ai_plan.shape}")
print(f"unique industries: {df_ai_plan['Industry'].nunique()}")

cleaned: (938, 6)
unique industries: 67


In [58]:
# AI actual usage (Q2 2025)
df_ai_actual = pd.read_csv(RAW_DATA / "33101004-eng" / "ai_actual_q2_2025.csv", encoding="utf-8")
# clean column names (remove quotes if any)
df_ai_actual.columns = df_ai_actual.columns.str.strip().str.replace('"', "")
print(f"AI actual: {df_ai_actual.shape}")

AI actual: (18760, 16)


In [59]:
ai_col2 = "Use of artificial intelligence (AI) by businesses or organizations in producing goods or delivering services over the last 12 months"

df_ai_actual["AI_Status"] = df_ai_actual[ai_col2].apply(
    lambda x: "Yes" if "Yes" in str(x) else ("No" if "No" in str(x) else "Unknown")
)

df_ai_actual["Industry"] = df_ai_actual["Business characteristics"].str.strip()

df_ai_actual = df_ai_actual[df_ai_actual["AI_Status"] == "Yes"][
    ["REF_DATE", "GEO", "Industry", "AI_Status", "VALUE"]
].copy()

df_ai_actual["VALUE"] = pd.to_numeric(df_ai_actual["VALUE"], errors="coerce")
df_ai_actual["Source"] = "Actual_Q2_2025"
print(f"cleaned: {df_ai_actual.shape}")
print(f"unique industries: {df_ai_actual['Industry'].nunique()}")

cleaned: (938, 6)
unique industries: 67


In [ ]:
# combine both AI datasets
df_ai = pd.concat([df_ai_plan, df_ai_actual], ignore_index=True)

# filter to only keep actual industries (those with NAICS codes in brackets like [72])
# this removes rows like "1 to 4 employees", "Age of business", etc.
df_ai = df_ai[df_ai["Industry"].str.contains(r"\[\d+\]", na=False, regex=True)].copy()

# clean industry names - remove NAICS codes in brackets
# "Accommodation and food services [72]" -> "Accommodation and food services"
df_ai["Industry_Clean"] = df_ai["Industry"].str.replace(r"\s*\[\d+\]", "", regex=True).str.strip()

# calculate average AI exposure per industry
# VALUE = % of businesses in that industry using/planning AI
df_ai_exposure = df_ai.groupby("Industry_Clean")["VALUE"].mean().reset_index()
df_ai_exposure.columns = ["Industry", "AI_Exposure_Score"]

print(f"calculated AI exposure for {len(df_ai_exposure)} industries")
print(
    f"AI score range: {df_ai_exposure['AI_Exposure_Score'].min():.1f}% to {df_ai_exposure['AI_Exposure_Score'].max():.1f}%"
)
df_ai_exposure.head(10)

calculated AI exposure for 13 industries
AI score range: 2.3% to 25.7%


,Industry,AI_Exposure_Score
0,Accommodation and food services,4.646429
1,"Administrative and support, waste management a...",8.410714
2,"Agriculture, forestry, fishing and hunting",2.251852
3,"Arts, entertainment and recreation",9.807407
4,Construction,2.885714
5,Finance and insurance,20.270370
6,Health care and social assistance,10.450000
7,Information and cultural industries,25.688000
8,"Mining, quarrying, and oil and gas extraction",5.851852
9,Other services (except public administration),5.022222


## Step 4: Map occupations to industries

Our wage data uses NOC codes (National Occupational Classification) but AI data uses NAICS codes (industries). We need to map job titles to industries using keyword matching.


In [61]:
# map job titles to industries using keywords
# we match job title words to NAICS industry categories (same names used in AI data)

keywords = {
    "Information and cultural industries": [
        "software",
        "developer",
        "programmer",
        "web",
        "data",
        "IT",
        "computer",
        "analyst",
    ],
    "Professional, scientific and technical services": [
        "engineer",
        "scientist",
        "researcher",
        "consultant",
        "architect",
        "designer",
    ],
    "Finance and insurance": ["accountant", "financial", "banker", "insurance", "auditor"],
    "Health care and social assistance": ["nurse", "doctor", "physician", "therapist", "medical", "health"],
    "Manufacturing": ["machinist", "assembler", "fabricator", "production", "manufacturing", "industrial"],
    "Retail trade": ["retail", "sales", "cashier", "store"],
    "Construction": ["carpenter", "electrician", "plumber", "construction", "contractor"],
    "Transportation and warehousing": ["driver", "pilot", "courier", "warehouse"],
    "Accommodation and food services": ["cook", "chef", "server", "bartender", "hotel"],
    "Educational services": ["teacher", "professor", "instructor", "education"],
}


def get_industry(title):
    if pd.isna(title):
        return "Other"

    title_lower = str(title).lower()
    for industry, words in keywords.items():
        if any(w.lower() in title_lower for w in words):
            return industry
    return "Other"


df_wages_clean["Industry"] = df_wages_clean["NOC_Title_Standardized"].apply(get_industry)

print("industry mapping done")
print(f"mapped {df_wages_clean['Industry'].nunique()} industries")
print("\ntop 5 industries by job count:")
print(df_wages_clean["Industry"].value_counts().head())

industry mapping done
mapped 11 industries

top 5 industries by job count:
Industry
Other                                              575684
Information and cultural industries                 11352
Professional, scientific and technical services      5504
Manufacturing                                        5160
Health care and social assistance                    4644
Name: count, dtype: int64


## Step 5: Merge wages with AI exposure

Now we combine the wage data with AI adoption scores to create our final master dataset.


In [62]:
# merge wages with AI exposure scores
df_master = df_wages_clean.merge(df_ai_exposure, on="Industry", how="left")

# fill missing AI scores with 0 (industries not in AI dataset)
df_master["AI_Exposure_Score"].fillna(0, inplace=True)

# categorize AI exposure as low/medium/high
# low = 0-5%, medium = 5-10%, high = 10%+
df_master["AI_Exposure_Category"] = pd.cut(
    df_master["AI_Exposure_Score"], bins=[0, 5, 10, 100], labels=["Low", "Medium", "High"]
)

print(f"final dataset: {df_master.shape}")
print(f"matched {(df_master['AI_Exposure_Score'] > 0).sum():,} rows with AI data")
print(f"\nAI score range: {df_master['AI_Exposure_Score'].min():.1f}% to {df_master['AI_Exposure_Score'].max():.1f}%")
print("\nAI exposure breakdown:")
print(df_master["AI_Exposure_Category"].value_counts())

final dataset: (613008, 17)
matched 27,176 rows with AI data

AI score range: 0.0% to 25.7%

AI exposure breakdown:
AI_Exposure_Category
High      23392
Low        3784
Medium        0
Name: count, dtype: int64


In [63]:
df_master.head()

,NOC_CNP,NOC_Title_Standardized,Province,Region,Reference_Period,Annual_Wage_Flag_Salaire_annuel,Low_Wage_Hourly,Median_Wage_Hourly,High_Wage_Hourly,Average_Wage_Hourly,Quartile1_Wage_Hourly,Quartile3_Wage_Hourly,missing_wage_flag,is_entry_level,Industry,AI_Exposure_Score,AI_Exposure_Category
0,NaN,NaN,NaN,NaN,NaN,0.0,8.17,28.00,49.45,NaN,NaN,NaN,0,0,Other,0.0,NaN
1,NaN,NaN,NaN,NaN,NaN,0.0,17.95,40.06,64.97,NaN,NaN,NaN,0,0,Other,0.0,NaN
2,NaN,NaN,NaN,NaN,NaN,0.0,19.23,45.67,66.67,NaN,NaN,NaN,0,0,Other,0.0,NaN
3,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Other,0.0,NaN
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Other,0.0,NaN


In [64]:
# check for missing data
print("missing values:")
print(df_master.isnull().sum())

missing values:
NOC_CNP                            527008
NOC_Title_Standardized             524256
Province                           480396
Region                             479880
Reference_Period                   576120
Annual_Wage_Flag_Salaire_annuel    129000
Low_Wage_Hourly                    385834
Median_Wage_Hourly                 383394
High_Wage_Hourly                   385814
Average_Wage_Hourly                542612
Quartile1_Wage_Hourly              580638
Quartile3_Wage_Hourly              580634
missing_wage_flag                       0
is_entry_level                          0
Industry                                0
AI_Exposure_Score                       0
AI_Exposure_Category               585832
dtype: int64


## Step 6: Save everything

Saving the cleaned master dataset plus supporting files.


In [65]:
# save master dataset
df_master.to_csv(MERGED_DATA / "master_dataset.csv", index=False)
print(f"saved master_dataset.csv - {df_master.shape[0]:,} rows, {df_master.shape[1]} cols")

# save AI lookup table
df_ai_exposure.to_csv(CLEANED_DATA / "ai_exposure_by_industry.csv", index=False)
print("saved ai_exposure_by_industry.csv")

saved master_dataset.csv - 613,008 rows, 17 cols
saved ai_exposure_by_industry.csv


In [66]:
# create metadata doc
meta = f"""# Master Dataset Metadata
Generated: {pd.Timestamp.now()}

## Summary
- Rows: {df_master.shape[0]:,}
- Columns: {df_master.shape[1]}
- Years: {df_master["Reference_Period"].min()} to {df_master["Reference_Period"].max()}
- Provinces: {df_master["Province"].nunique()}
- NOC codes: {df_master["NOC_CNP"].nunique()}
- Occupations: {df_master["NOC_Title_Standardized"].nunique()}

## Sources
1. Job Bank wage data (2012-2024) - {len(df_wages_clean):,} rows
2. AI planned Q3 2024 - {len(df_ai_plan):,} rows  
3. AI actual Q2 2025 - {len(df_ai_actual):,} rows

## What we did
1. Converted annual wages to hourly (2080 hrs/year)
2. Flagged missing wage data
3. Standardized job titles
4. Merged similar titles (software engineer -> software developer)
5. Flagged entry level jobs (bottom 25% wages)
6. Mapped jobs to industries using keywords
7. Calculated AI exposure per industry
8. Merged wages with AI data
9. Created AI exposure categories

## Columns
- NOC_CNP: occupation code
- NOC_Title_Standardized: job title
- Province: province code
- Region: economic region
- Industry: mapped industry
- Reference_Period: year
- Low/Median/High/Average/Q1/Q3 Wage_Hourly: wage stats
- missing_wage_flag: 1 if all wages missing
- is_entry_level: 1 if bottom 25% wage
- AI_Exposure_Score: % businesses using AI
- AI_Exposure_Category: Low/Medium/High

## Notes
- {df_master["missing_wage_flag"].sum():,} rows have no wage data
- NOC to industry mapping is approximate (keyword based)
- Entry level = wages <= 25th percentile for that occupation
"""

with open(METADATA / "merge_log.md", "w") as f:
    f.write(meta)

print("saved merge_log.md")

saved merge_log.md


## Done!

Cleaned data saved to:

- data/merged/master_dataset.csv
- data/cleaned/ai_exposure_by_industry.csv
- data/metadata/merge_log.md

Next: run 02_analysis.ipynb
